In [290]:
from datetime import date
import uuid
import jaydebeapi, os
import random


def getToday():
    return date.today().isoformat()

def generateSQL(tablename, columns, values):
    
    sql= ' insert into ' + tableName + ' (' + columns +  ' ) values (' + values + ');'
    print(sql)
    return sql

In [291]:
class Customer:
    
    columns = {'customerNumber',
     'customerName',           
    'phoneNumber',
    'postalCode',
    'locale',
    'dateCreated',
    'email'}

    
    def __init__(self):
        
        self.customerNumber = uuid.uuid1()
        self.customerName = randomString(20)
        self.phoneNumber = random.randint(1000000,9000000)
        self.postalCode = randomString(4)
        self.locale = 'en-US.utf-8'
        self.dateCreated = getToday()
        self.email = randomString(4) + '@' + randomString(4) + ".com"
        
    def getColumns(self):
        return self.columns
    
    def getValues(self):
               
        return [self.customerNumber,
            self.customerName,
            self.phoneNumber,
            self.postalCode,
            self.locale,
            self.dateCreated,
            self.email]
     
    
'''    
create table customers (
customerNumber uuid,
customerName varchar(50),
phoneNumber varchar(14),
postalCode varchar(4),
locale varchar(11),
dateCreated timestamp,
email varchar(20));
'''        
        

#class InventoryMovement:
    

#class Product:
    

'''
class SalesOrder:

    self.customerNumber
    self.orderNumber
    self.comments
    self.orderDate
    self.orderType
    self.shipDate
    self.discount
    self.quantity
    self.productNumber
'''    

'''
create table salesOrder (
    customerNumber uuid,
    orderNumber uuid,
    comments varchar(200),
    orderDate timestamp,
    orderType varchar(20),
    shipDate timestamp,
    discount float,
    quantity integer,
    productNumber varchar(50));
'''


'\ncreate table salesOrder (\n    customerNumber uuid,\n    orderNumber uuid,\n    comments varchar(200),\n    orderDate timestamp,\n    orderType varchar(20),\n    shipDate timestamp,\n    discount float,\n    quantity integer,\n    productNumber varchar(50));\n'

In [292]:


def connect():
    dsn_database = "inventory"
    dsn_hostname = "paris2x"
    dsn_port = "5432"
    dsn_uid = "walker"
    dsn_pwd = "livres"
    jdbc_driver_name = "org.postgresql.Driver"
    jdbc_driver_loc = os.path.join('/home/ubuntu/Documents/postgresql-42.2.14.jar')

    sql_str = "select version()"

    connection_string='jdbc:postgresql://'+ dsn_hostname+':'+ dsn_port +'/'+ dsn_database

    url = '{0}:user={1};password={2}'.format(connection_string, dsn_uid, dsn_pwd)
    print("Connection String: " + connection_string)

    conn = jaydebeapi.connect(jdbc_driver_name, connection_string, {'user': dsn_uid, 'password': dsn_pwd},
    jars=jdbc_driver_loc)

    return conn


In [293]:


def randomString(length):
    char=""
    for i in range(length):
        char = char + chr(random.randint(65,65+25))
    return char

In [294]:
def commaCheck(field):
    
     print("type=", type(field))
    
     if isinstance(field, uuid.UUID):
        return   str(field)  
    
     if isinstance(field, int):
        return  str(field)  
    
     if isinstance(field, str):
        return field 

In [295]:
def generateCustomers(conn):
    for i in range(5):
        
        customer = Customer()
        columns = ','.join([col for col in customer.getColumns()])
        
        values = ','.join(["'" + commaCheck(value) + "'" for value in customer.getValues()])

        sql=generateSQL("customers",columns,values)

        curs = conn.cursor()
        curs.execute(sql)
        result = curs.fetchall()
        print(result)
        

In [296]:
def main():
    conn=connect()
    generateCustomers(conn)
   


if __name__ == "__main__":
    main()

Connection String: jdbc:postgresql://paris2x:5432/inventory
type= <class 'uuid.UUID'>
type= <class 'str'>
type= <class 'int'>
type= <class 'str'>
type= <class 'str'>
type= <class 'str'>
type= <class 'str'>
 insert into customers (phoneNumber,postalCode,customerNumber,locale,email,dateCreated,customerName ) values ('2e9cb3fe-e3ad-11ea-ab7a-0ec120e133fc','MEQSENHOCPTORFETMLIO','4016663','EUPI','en-US.utf-8','2020-08-21','MREH@ARET.com');


DatabaseError: org.postgresql.util.PSQLException: ERROR: invalid input syntax for uuid: "4016663"
  Position: 173

In [ ]:
str(uuid.uuid1())